# Que vamos a hacer:
### 3 Sistemas de recomendación
- Afinidad a Marca por artistas
- Afinidad a Marca por géneros
- Afinidad a Marca por subgéneros

- Tenemos que tener en cuenta que el usuario puede que no coincida con nada
- Que los rankings pueden ser de distinta longitud (uno mas largo que el otro)
- Que el numero de apariciones no podemos usarlo, debemos usar como peso la posicion del ranking que tenga, pues el numero de apariciones setea un peso distinto e injusto (cantidad de muestras en marca contra solo uno)

In [1]:
import sys
sys.path.append("../")
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")

from supabase import create_client, Client
from dotenv import load_dotenv
# Cargar variables de entorno
load_dotenv()

# Configuración de Supabase
url = os.getenv("project_url")
key = os.getenv("browser_safe_key")
supabase: Client = create_client(url, key)

In [2]:
def get_brand_artist_ranking(supabase_credential,brand_id):
    brand_ranking = supabase_credential.table("artists_ranking").select("artist_name","number_of_appearances").eq("brand_id",brand_id).order("number_of_appearances",desc=True).execute().data
    artistas = []
    apariciones = []
    for ranking in brand_ranking:
        artistas.append(ranking["artist_name"])
        apariciones.append(ranking["number_of_appearances"])

    brand_df = pd.DataFrame({
        "artista" : artistas,
        "apariciones" : apariciones
    })

    brand_df.index = brand_df.index + 1
    brand_df.reset_index(inplace=True)
    brand_df.columns = ["ranking","artista","apariciones"]
    brand_df.drop(columns="apariciones",inplace=True)
    brand_df["artista"] = brand_df["artista"].astype(str)
    return brand_df

brand_df = get_brand_artist_ranking(supabase,1)

In [3]:
def get_user_artist_ranking(supabase_credential,user_id):
    user_ranking = supabase_credential.table("user_artists_ranking").select("artist_name","number_of_appearances").eq("user_id",user_id).order("number_of_appearances",desc=True).execute().data
    artistas = []
    apariciones = []
    for ranking in user_ranking:
        artistas.append(ranking["artist_name"])
        apariciones.append(ranking["number_of_appearances"])

    user_df = pd.DataFrame({
        "artista" : artistas,
        "apariciones" : apariciones
    })

    user_df.index = user_df.index + 1
    user_df.reset_index(inplace=True)
    user_df.columns = ["ranking","artista","apariciones"]
    user_df.drop(columns="apariciones",inplace=True)
    user_df["artista"] = user_df["artista"].astype(str)
    return user_df

user_df = get_user_artist_ranking(supabase,"gonzaloruiperez")


In [ ]:
def obtener_afinidad_por_artista(supabase_credential,brand_id,user_id):
    
    # Obtener los rankings de la marca y el usuario
    brand_df = get_brand_artist_ranking(supabase_credential, brand_id)
    user_df = get_user_artist_ranking(supabase_credential, user_id)
    
    # Obtener los artistas en común
    artistas_comunes = set(user_df['artista']).intersection(set(brand_df['artista']))
    user_df_filtrado = user_df[user_df['artista'].isin(artistas_comunes)]
    brand_df_filtrado = brand_df[brand_df['artista'].isin(artistas_comunes)]

    # Reseteamos el Index
    user_df_filtrado.reset_index(drop=True, inplace=True)
    brand_df_filtrado.reset_index(drop=True, inplace=True)

    # Asignar pesos en base a la posición en el ranking
    user_df_filtrado["peso"] = 1 / user_df_filtrado["ranking"]
    brand_df_filtrado["peso"] = 1 / brand_df_filtrado["ranking"]

    # Normalizar los pesos para evitar sesgos por diferencias de tamaño
    user_df_filtrado["peso"] /= user_df_filtrado["peso"].sum()
    brand_df_filtrado["peso"] /= brand_df_filtrado["peso"].sum()

    # Poner como index artista y peso se queda como columna
    pesos_user = user_df_filtrado.set_index("artista")["peso"]
    pesos_brand = brand_df_filtrado.set_index("artista")["peso"]

    # Ordenamos los artistas por orden alfabético para que los vectores estén alineados
    sorted_pesos = sorted(pesos_user.index) 

    # Generamos los vectores
    user_vector = np.array([pesos_user[a] for a in sorted_pesos])
    brand_vector = np.array([pesos_brand[a] for a in sorted_pesos])

    # Crear matriz de comparación
    matriz_pesos = np.vstack([user_vector, brand_vector])

    # Calcular la matriz de distancias con pdist
    matriz_distancias = squareform(pdist(matriz_pesos, metric="euclidean"))

    # Extraer la distancia entre usuario y marca
    distancia = matriz_distancias[0, 1]

    # Convertimos la distancia en afinidad (invirtiendo la escala)
    afinidad = max(0, (1 - distancia) * 100)

    afinidad

In [26]:
# Obtener los artistas en común
artistas_comunes = set(user_df['artista']).intersection(set(brand_df['artista']))
user_df_filtrado = user_df[user_df['artista'].isin(artistas_comunes)]
brand_df_filtrado = brand_df[brand_df['artista'].isin(artistas_comunes)]

# Reseteamos el Index
user_df_filtrado.reset_index(drop=True, inplace=True)
brand_df_filtrado.reset_index(drop=True, inplace=True)

# Asignar pesos en base a la posición en el ranking
user_df_filtrado["peso"] = 1 / user_df_filtrado["ranking"]
brand_df_filtrado["peso"] = 1 / brand_df_filtrado["ranking"]

# Normalizar los pesos para evitar sesgos por diferencias de tamaño
user_df_filtrado["peso"] /= user_df_filtrado["peso"].sum()
brand_df_filtrado["peso"] /= brand_df_filtrado["peso"].sum()

# Poner como index artista y peso se queda como columna
pesos_user = user_df_filtrado.set_index("artista")["peso"]
pesos_brand = brand_df_filtrado.set_index("artista")["peso"]

# Ordenamos los artistas por orden alfabético para que los vectores estén alineados
sorted_pesos = sorted(pesos_user.index) 

# Generamos los vectores
user_vector = np.array([pesos_user[a] for a in sorted_pesos])
brand_vector = np.array([pesos_brand[a] for a in sorted_pesos])

# Crear matriz de comparación
matriz_pesos = np.vstack([user_vector, brand_vector])

# Calcular la matriz de distancias con pdist
matriz_distancias = squareform(pdist(matriz_pesos, metric="euclidean"))

# Extraer la distancia entre usuario y marca
distancia = matriz_distancias[0, 1]

# Convertimos la distancia en afinidad (invirtiendo la escala)
afinidad = max(0, (1 - distancia) * 100)

float(round(afinidad,2))

70.37

# 1 Buscamos los artistas comunes
Dejando unicamente aquellos que coinciden con su posición en cada ranking!

In [5]:
# Obtener los artistas en común
artistas_comunes = set(user_df['artista']).intersection(set(brand_df['artista']))
user_df_filtrado = user_df[user_df['artista'].isin(artistas_comunes)]
brand_df_filtrado = brand_df[brand_df['artista'].isin(artistas_comunes)]

user_df_filtrado.reset_index(drop=True, inplace=True)
brand_df_filtrado.reset_index(drop=True, inplace=True)

# 2 Asignamos pesos según su posición en el ranking 

- Fórmula utilizada
$$
peso = \frac{1}{\text{posición}}
$$

No buscamos complejidad, buscamos que funcione de momento


In [6]:
# Asignar pesos en base a la posición en el ranking
user_df_filtrado["peso"] = 1 / user_df_filtrado["ranking"]
brand_df_filtrado["peso"] = 1 / brand_df_filtrado["ranking"]

# 3 Normalizamos los pesos
- Dado que la longitud y ranking es distinto, debemos normalizar los pesos a la misma escala
- De esta forma nos aseguramos de que el cálculo de la afinidad es justo
- Fórmula utilizada

$$
peso\_normalizado = \frac{peso\_original}{\sum peso\_original}
$$

- Ahora los pesos están a la misma escala

### Operador /=
- Es lo mismo que
```python
    df_user_filtrado["peso"] = df_user_filtrado["peso"] / df_user_filtrado["peso"].sum()
```

- Esto garantiza que todos los pesos sumen 1, permitiendo comparaciones justas entre rankings de distinto tamaño.
- Normalizar los pesos evita que rankings más largos o cortos afecten injustamente el cálculo de afinidad.


In [7]:
# Normalizar los pesos para evitar sesgos por diferencias de tamaño
user_df_filtrado["peso"] /= user_df_filtrado["peso"].sum()
brand_df_filtrado["peso"] /= brand_df_filtrado["peso"].sum()

# 4 Poner como index artista y peso se queda como columna

In [8]:
# Asegurar que los artistas están alineados en el mismo orden
pesos_usuario = user_df_filtrado.set_index("artista")["peso"]
pesos_zara = brand_df_filtrado.set_index("artista")["peso"]

# 5 Ordenamos los artistas para que los vectores estén alineados

In [9]:
# Ordenamos los artistas para que los vectores estén alineados
artistas_ordenados = sorted(pesos_usuario.index)  # Orden alfabético
vector_usuario = np.array([pesos_usuario[a] for a in artistas_ordenados])
vector_zara = np.array([pesos_zara[a] for a in artistas_ordenados])

# 6 Creamos Matriz de Comparación

In [10]:
# Crear matriz de comparación
matriz_pesos = np.vstack([vector_usuario, vector_zara])

# 7 Calculamos las distancias con pdist y squareform

In [11]:
# Calcular la matriz de distancias con pdist
matriz_distancias = squareform(pdist(matriz_pesos, metric="euclidean"))

# 8 Extraemos distancias

In [12]:
# Extraer la distancia entre usuario y Zara
distancia = matriz_distancias[0, 1]

# 9 Obtenemos Porcentaje de Afinidad

In [13]:
# Convertimos la distancia en afinidad (invirtiendo la escala)
afinidad = max(0, (1 - distancia) * 100)

In [14]:
afinidad

np.float64(70.37332110089116)

# Esto lo uso para ver que métricas dan

In [15]:
metricas = ["braycurtis", "chebyshev", "cosine", "euclidean","jensenshannon", "minkowski"]

for metrica in metricas:
    # Calcular la matriz de distancias con pdist
    matriz_distancias = squareform(pdist(matriz_pesos, metric=metrica))
    # Extraer la distancia entre usuario y Zara
    distancia = matriz_distancias[0, 1]
    # Convertimos la distancia en afinidad (invirtiendo la escala)
    afinidad = max(0, (1 - distancia) * 100)
    print(f"Afinidad con {metrica}: {afinidad}%")

Afinidad con braycurtis: 26.121723454494315%
Afinidad con chebyshev: 80.88734399546608%
Afinidad con cosine: 11.255790055954595%
Afinidad con euclidean: 70.37332110089116%
Afinidad con jensenshannon: 36.71152212714518%
Afinidad con minkowski: 70.37332110089116%


In [16]:
metricas = ["braycurtis", "canberra", "chebyshev", "cityblock", "correlation", "cosine",
             "dice", "euclidean", "hamming", "jaccard", "jensenshannon", "kulczynski1",
                "matching", "minkowski", "rogerstanimoto", "russellrao",
                 "seuclidean", "sokalmichener", "sokalsneath", "sqeuclidean", "yule"]

for metrica in metricas:
    # Calcular la matriz de distancias con pdist
    matriz_distancias = squareform(pdist(matriz_pesos, metric=metrica))
    # Extraer la distancia entre usuario y Zara
    distancia = matriz_distancias[0, 1]
    # Convertimos la distancia en afinidad (invirtiendo la escala)
    afinidad = max(0, (1 - distancia) * 100)
    print(f"Afinidad con {metrica}: {afinidad}%")

Afinidad con braycurtis: 26.121723454494315%
Afinidad con canberra: 0%
Afinidad con chebyshev: 80.88734399546608%
Afinidad con cityblock: 0%
Afinidad con correlation: 9.112750596419872%
Afinidad con cosine: 11.255790055954595%
Afinidad con dice: 0.5349848238353938%
Afinidad con euclidean: 70.37332110089116%
Afinidad con hamming: 0%
Afinidad con jaccard: 100.0%
Afinidad con jensenshannon: 36.71152212714518%
Afinidad con kulczynski1: 0%
Afinidad con matching: 0%
Afinidad con minkowski: 70.37332110089116%
Afinidad con rogerstanimoto: 100.0%
Afinidad con russellrao: 100.0%
Afinidad con seuclidean: 0%
Afinidad con sokalmichener: 100.0%
Afinidad con sokalsneath: 100.0%
Afinidad con sqeuclidean: 91.22259897409099%
Afinidad con yule: 100.0%
